In [1]:
import pandas as pd
import numpy as np
import requests, math, os, yaml

### Define Helper Functions

In [2]:
def drugmech_path_to_str(path, types_only=False, simplify_mechs=False):
    if types_only:
        node_names = [node['label'] for node in path['nodes']]
        if simplify_mechs:
            node_names = [simplify_mech(name) for name in node_names]
    else:
        node_names = [node['name'] for node in path['nodes']]
        if simplify_mechs:
            print('Warning: simplify_mechs not used when types_only=False')
        
        
    rel_names = [link['key'] for link in path['links']]
    hop_length = len(rel_names)
    assert len(node_names) == hop_length + 1
    s = ''
    for i in range(hop_length):
        s += f'({node_names.pop(0)})'
        s += f'--{rel_names.pop(0)}--'
    s += node_names.pop(0)
    return s

def simplify_mech(node_label):
    if node_label in ['BiologicalProcess', 'MolecularActivity', 'Pathway']:
        out = 'Mechanism'
    else:
        out = node_label
    return out

def get_key_value_pair_from_string(s):

    key = None
    value = None

    if ': ' in s:
        toks = s.split(': ')
        key = toks[0]
        value = ''.join(toks[1:])
        value = value.rstrip()
    else:
        print(f'Warning: Unexpected string value: {s}')

    return key, value


def get_mesh_to_disease_ontology(do_file = '../data/disease_ontology/doid.obo.txt', verbose=False):

    #do_to_mesh = dict()
    mesh_to_do = dict()

    with open(do_file, 'r') as f:

        # burn through header
        line = f.readline()
        while line != '[Term]\n':
            line = f.readline()

        # go through each term
        while line:

            line = f.readline()

            do_ids = []
            mesh_ids = []
            
            while line not in ['[Term]\n', '', '[Typedef]\n', '\n']:

                key, value = get_key_value_pair_from_string(line)

                if key == 'id' or key == 'alt_id':
                    #do_ids.append('Disease::' + value)
                    do_ids.append(value)

                if key == 'xref' and value.startswith('MESH'):
                    #print('appending mesh id')
                    mesh_ids.append(value)

                line = f.readline()
            
            if len(do_ids) >= 1 and len(mesh_ids) >= 1:
                #do_to_mesh[do_ids[0]] = mesh_ids
                for mesh in mesh_ids:
                    #if len(do_ids) > 1:
                    #    if verbose: print('warning: only mapping to first of multiple DO ids')
                    if mesh in mesh_to_do.keys():
                        mesh_to_do[mesh].extend(do_ids)
                    else:
                        mesh_to_do[mesh] = do_ids
    
    # Remove duplicates
    nodup_mesh_to_do = dict()
    for mesh, do_list in mesh_to_do.items():
        n1 = len(do_list)
        n2 = len(set(do_list))
        if n1 - n2 > 0:
            print(f'Removed {n1-n2} duplicate DO terms from {mesh} mapping')
        nodup_mesh_to_do[mesh] = list(set(do_list))
        
    return nodup_mesh_to_do
    
    
def get_mesh_to_drugbank(drop_duplicates=True):
    # Using DrugCentral
    # Code pulled from https://github.com/dhimmel/clintrials/blob/4d63098c79042b7048f546720e727bc94e232182/2.map.ipynb
    url = 'https://github.com/olegursu/drugtarget/blob/9a6d84bed8650c6c507a2d3d786814c774568610/identifiers.tsv?raw=true'
    drug_map_df = pd.read_table(url)
    drug_map_df = drug_map_df[drug_map_df.ID_TYPE.str.contains('MESH')][['DRUG_ID', 'IDENTIFIER']].rename(columns={'IDENTIFIER': 'intervention'}).merge(
    drug_map_df[drug_map_df.ID_TYPE == 'DRUGBANK_ID'][['DRUG_ID', 'IDENTIFIER']]
    ).drop('DRUG_ID', axis='columns')
    drug_map_df.rename(columns={'intervention': 'mesh_id', 'IDENTIFIER': 'drugbank_id'}, inplace=True)
    drug_map_df['mesh_id'] = 'MESH:' + drug_map_df['mesh_id']
    
    if drop_duplicates:
        # drop duplicates for each column separately to ensure that there is a 1:1 mapping between ids
        drug_map_df.drop_duplicates(subset=['mesh_id'], inplace=True)
        drug_map_df.drop_duplicates(subset=['drugbank_id'], inplace=True)
    
    drug_map_df.set_index('mesh_id', drop=True, inplace=True)
    return drug_map_df

def map_id_to_hetnet_drugbank(drug_id, mesh_to_drugbank, verbose=False):
    if drug_id.startswith('MESH'):
        if drug_id in mesh_to_drugbank.index:
            db_id = mesh_to_drugbank['drugbank_id'][drug_id]
            out = f'Compound::{db_id}'
        else:
            out = math.nan
            if verbose: print(f'Warning: No mapping from {drug_id} to drugbank id found')
    else:
        assert drug_id.startswith('DB')
        out = f'Compound::{drug_id}'
    return out

def get_uniprot_to_entrez(uniprot_list, abbr_from='ACC+ID', abbr_to='P_ENTREZGENEID', drop_duplicates=True): 
    #For more info: https://www.uniprot.org/help/api_idmapping
    
    uniprot_str = ' '.join(uniprot_list)

    url = 'https://www.uniprot.org/uploadlists/'

    params = {
    'from': abbr_from,
    'to': abbr_to,
    'format': 'tab',
    'query': uniprot_str
    }

    response = requests.get(url, params)

    # postprocess output into dataframe
    data = response.content.decode('utf-8')
    rows = [row.split('\t') for row in data.split('\n')]
    _ = rows.pop(0)
    _ = rows.pop()
    uniprot_to_entrez = pd.DataFrame(rows, columns=['protein_id_dmdb', 'gene_id_entrez'])
    uniprot_to_entrez['protein_id_dmdb'] = 'UniProt:' + uniprot_to_entrez['protein_id_dmdb']
    uniprot_to_entrez['gene_id_hetnet'] = 'Gene::' + uniprot_to_entrez['gene_id_entrez']
    
    if drop_duplicates:
        uniprot_to_entrez.drop_duplicates(subset='protein_id_dmdb', keep=False, inplace=True)
    
    return uniprot_to_entrez

def map_dmdb_disease_to_hetnet_disease_or_symptom(mesh, mesh_to_do, hetnet_nodes, all_hetnet_ids):

    found_in_hetnet = False
    hetnet_id = np.nan
    hetnet_type = np.nan
    
    if mesh in mesh_to_do.keys():
        
        do_ids = mesh_to_do[mesh]
    
        matched = 0
        do_matched = None
        for i, do in enumerate(do_ids):
            if do in all_hetnet_ids:
                found_in_hetnet = True
                matched +=1
                do_matched = do
        
        if found_in_hetnet:
            if matched > 1:
                print(f'Warning: {matched} DO ids are in Hetnet for {mesh}')
            else:
                assert matched == 1
                hetnet_id = 'Disease::' + do_matched
                hetnet_type = 'Disease'
                
    # If we didn't find it in disease, check in symptom
    if not found_in_hetnet:
        if mesh[5:] in all_hetnet_ids:
            found_in_hetnet=True
            hetnet_id = 'Symptom::' + mesh[5:]
            hetnet_type = 'Symptom'
                
    return hetnet_id, hetnet_type


def merge_and_count(df_left, df_right, description='', **merge_args):
    n_start = df_left.shape[0]
    df_left = df_left.merge(df_right, **merge_args)
    n_end = df_left.shape[0]
    print(f'Lost {n_start - n_end} rows when merging {description}')
    return df_left 

def subset_and_count(df, column_name, allowed_values, description=''):
    assert column_name in df.columns

    n_start = df.shape[0]
    df = df[df[column_name].isin(allowed_values)]
    n_end = df.shape[0]
    
    print(f'Lost {n_start - n_end} rows when subsetting {column_name} {description}')
    return df

### Read in Hetnet nodes and add column with simplified node id

In [3]:
hetnet_nodes = pd.read_csv('../data/hetnet/hetionet-v1.0-nodes.tsv', sep='\t')

stripped_ids = []
for i, row in hetnet_nodes.iterrows():
    stripped_ids.append(row.id.split(row.kind + '::')[1])
hetnet_nodes['id_stripped'] = stripped_ids

### Read in DrugMechDB and filter to relevant subset

In [4]:
# Read DrugMechDB Dataset
with open("../data/drugmechdb/DrugMechDB_indication_paths.yml", 'r') as stream:
    try:
        drugmech = (yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
# Apply various filters to data
single_protein, protein_start = 0, 0
linear_paths = 0
signed_relationships = 0
human_only = 0
idx_to_keep = []

for i, graph in enumerate(drugmech):
    labels = [node['label'] for node in graph['nodes']]
    rels = [link['key'] for link in graph['links']]

    if len(labels) == len(graph['links']) + 1:
        linear_paths += 1
    
        if labels.count('Protein') == 1: 
            single_protein += 1

            if labels[1] == 'Protein':
                protein_start += 1
        
                if 'OrganismTaxon' not in labels:
                    human_only += 1
            
                    if rels[0] in ['decreases activity of','increases activity of', 'negatively regulates', 'positively regulates']:
                        signed_relationships += 1
                        
                        idx_to_keep.append(i)

drugmech_subset = [drugmech[i] for i in idx_to_keep]

print(f'Total number of explanation graphs from DrugMechDB: {len(drugmech)}')
print(f'Number of these that involve only a single, linear path: {linear_paths}')
print(f'Number of these that involve only a single protein: {single_protein}')
print(f'Number of these where the protein is directly linked to the drug: {protein_start}')
print(f'Number of these that do not mention other organism taxons: {human_only}')
print(f'Number of these where the Drug-Protein relationship has a clear sign: {signed_relationships}')

Total number of explanation graphs from DrugMechDB: 1245
Number of these that involve only a single, linear path: 861
Number of these that involve only a single protein: 486
Number of these where the protein is directly linked to the drug: 465
Number of these that do not mention other organism taxons: 372
Number of these where the Drug-Protein relationship has a clear sign: 357


### Construct dataframe from relevant aspects of drugmechdb data

In [6]:
rows = []
for i, path in enumerate(drugmech_subset):
  
    # extract drug and corresponding ids
    drug_name = path['nodes'][0]['name']
    drug_id_dmdb = path['nodes'][0]['id']
    
    # extract gene and corresponding ids
    protein_name = path['nodes'][1]['name']
    protein_id_dmdb = path['nodes'][1]['id']
    
    # extract disease and corresponding ids
    disease_name = path['nodes'][-1]['name']
    disease_id_dmdb = path['nodes'][-1]['id']
    
    # extract relationship 
    rel_drugmech = path['links'][0]['key']
    if rel_drugmech in ['negatively regulates', 'decreases activity of']:
        rel_hetnet = 'downregulates'
    elif rel_drugmech in ['positively regulates', 'increases activity of']:
        rel_hetnet = 'upregulates'
    else:
        print(f'Warning: rel_drugmech has unexpected value: {rel_drugmech}')
        
    rows.append({
        'drug_name': drug_name,
        'drug_id_dmdb': drug_id_dmdb,
        'protein_name': protein_name,
        'protein_id_dmdb': protein_id_dmdb,
        'disease_name': disease_name,
        'disease_id_dmdb': disease_id_dmdb,
        'relation_1': rel_hetnet,
        'relation_2': 'associates'
    })
    
drugmech_df = pd.DataFrame(rows)
drugmech_df.drop_duplicates(inplace=True, ignore_index=True)
drugmech_df

,drug_name,drug_id_dmdb,protein_name,protein_id_dmdb,disease_name,disease_id_dmdb,relation_1,relation_2
0,imatinib,MESH:D000068877,BCR/ABL,UniProt:P00519,CML (ph+),MESH:D015464,downregulates,associates
1,naproxen,MESH:D009288,Cox-2,UniProt:P35354,Pain,MESH:D010146,downregulates,associates
2,pinacidil,MESH:D020110,Calcium-activated potassium channel subunit al...,UniProt:Q12791,Hypertension,MESH:D006973,upregulates,associates
3,posaconazole,MESH:C101425,sterol 14a-demethylase,UniProt:P10613,Fungal Infection,MESH:D009181,downregulates,associates
4,antazoline,MESH:D000865,histamine H1 Receptor,UniProt:P35367,Vasomotor rhinitis,MESH:D012223,downregulates,associates
...,...,...,...,...,...,...,...,...
325,Levonorgestrel,MESH:D016912,Progesterone receptor,UniProt:P06401,Menorrhagia,MESH:D008595,upregulates,associates
326,Norgestrel,MESH:D009644,Progesterone receptor,UniProt:P06401,Menorrhagia,MESH:D008595,upregulates,associates
327,Clofedanol,MESH:C010432,Histamine H1 receptor,UniProt:P35367,Allergic rhinitis,MESH:D065631,downregulates,associates
328,Clofedanol,MESH:C010432,Histamine H1 receptor,UniProt:P35367,Rhinitis,MESH:D012220,downregulates,associates


### Map DrugMechDB proteins to HetNet Genes

In [7]:
# Map from Uniprot to Entrez
uniprot_list = [p[8:] for p in drugmech_df['protein_id_dmdb']]
uniprot_to_entrez = get_uniprot_to_entrez(uniprot_list, abbr_from='ACC+ID')
drugmech_df = merge_and_count(df_left=drugmech_df, df_right=uniprot_to_entrez[['protein_id_dmdb', 'gene_id_hetnet']],
                              on='protein_id_dmdb', how='inner', description='entrez gene ids to uniprot ids')

Lost 5 rows when merging entrez gene ids to uniprot ids


In [8]:
# Filter to Entrez ids in Hetnet
drugmech_df = subset_and_count(drugmech_df, 
                               column_name='gene_id_hetnet',
                               allowed_values=list(hetnet_nodes.id), 
                               description='to HetNet gene ids')

Lost 2 rows when subsetting gene_id_hetnet to HetNet gene ids


### Map DrugMechDB Drugs to HetNet Compounds

In [9]:
# Map to drugbank ids
n_start = drugmech_df.shape[0]

mesh_to_drugbank = get_mesh_to_drugbank(drop_duplicates=True)
hetnet_ids = [map_id_to_hetnet_drugbank(drug_id, mesh_to_drugbank) for drug_id in drugmech_df['drug_id_dmdb']]
drugmech_df['drug_id_hetnet'] = hetnet_ids
drugmech_df.dropna(subset=['drug_id_hetnet'], inplace=True)

n_end = drugmech_df.shape[0]
print(f'Lost {n_start - n_end} rows when mapping to drugbank ids')

Lost 52 rows when mapping to drugbank ids


In [10]:
# Subset to those in the HetNet
drugmech_df = subset_and_count(drugmech_df,
                              column_name='drug_id_hetnet',
                              allowed_values=list(hetnet_nodes.id),
                              description='to HetNet compound ids')

Lost 62 rows when subsetting drug_id_hetnet to HetNet compound ids


### Map DrugMechDB Diseases to HetNet Diseases and Symptoms

In [11]:
mesh_to_do = get_mesh_to_disease_ontology()
all_hetnet_ids = set(hetnet_nodes['id_stripped'])

hetnet_ids, hetnet_types = [], []
for _, mech in drugmech_df.iterrows():
    mesh = mech.disease_id_dmdb
    h_id, h_type = map_dmdb_disease_to_hetnet_disease_or_symptom(mesh, mesh_to_do, hetnet_nodes, all_hetnet_ids)
    hetnet_ids.append(h_id)
    hetnet_types.append(h_type)
drugmech_df['outcome_id_hetnet'] = hetnet_ids
drugmech_df['outcome_type_hetnet'] = hetnet_types

Removed 1 duplicate DO terms from MESH:C562741 mapping
Removed 1 duplicate DO terms from MESH:D050398 mapping


In [12]:
n_start = drugmech_df.shape[0]
drugmech_df.dropna(subset=['outcome_id_hetnet', 'outcome_type_hetnet'], inplace=True)
drugmech_df.reset_index(drop=True, inplace=True)
n_end = drugmech_df.shape[0]
print(f'Lost {n_start - n_end} rows mapping DrugMechDB disease ids to Hetnet Disease and Symptoms')

Lost 128 rows mapping DrugMechDB disease ids to Hetnet Disease and Symptoms


### Format outputs and save

In [13]:
out_dir = '../data/biomedical_benchmark'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    
drugmech_df.to_csv(f'{out_dir}/drugmechdb_hetnet_alldata.tsv', sep='\t', index=False)
    
for outcome, df in drugmech_df.groupby('outcome_type_hetnet'):
    cols = ['drug_id_hetnet', 'relation_1', 'gene_id_hetnet', 'relation_2', 'outcome_id_hetnet']
    df[cols].to_csv(f'{out_dir}/drugmechdb_hetnet_{outcome.lower()}_explanations.tsv', sep='\t', index=False)
    
    cols = ['drug_name', 'relation_1', 'protein_name', 'relation_2', 'disease_name']
    df[cols].to_csv(f'{out_dir}/drugmechdb_hetnet_{outcome.lower()}_explanations_names.tsv', sep='\t', index=False)

### Inspect output

In [14]:
num_drugs = len(set(drugmech_df.drug_name))
num_diseases = len(set(drugmech_df.disease_name))
num_proteins = len(set(drugmech_df.protein_name))

In [15]:
print(f'Number of unique entities: {num_drugs} drugs, {num_diseases} diseases and {num_proteins} proteins')

Number of unique entities: 47 drugs, 38 diseases and 51 proteins


In [16]:
num_drug_disease = drugmech_df[['drug_name', 'disease_name']].value_counts().shape[0]
print(f'Unique drug-disease pairs: {num_drug_disease}') 

Unique drug-disease pairs: 61


In [17]:
drugmech_df['relation_1'].value_counts()

downregulates    60
upregulates      21
Name: relation_1, dtype: int64

In [18]:
drugmech_df

,drug_name,drug_id_dmdb,protein_name,protein_id_dmdb,disease_name,disease_id_dmdb,relation_1,relation_2,gene_id_hetnet,drug_id_hetnet,outcome_id_hetnet,outcome_type_hetnet
0,naproxen,MESH:D009288,Cox-2,UniProt:P35354,Pain,MESH:D010146,downregulates,associates,Gene::5743,Compound::DB00788,Symptom::D010146,Symptom
1,mepyramine,MESH:D011738,Histamine H1 receptor,UniProt:P35367,Sneezing,MESH:D012912,downregulates,associates,Gene::3269,Compound::DB06691,Symptom::D012912,Symptom
2,mepyramine,MESH:D011738,Histamine H1 receptor,UniProt:P35367,Dysmenorrhea,MESH:D004412,downregulates,associates,Gene::3269,Compound::DB06691,Symptom::D004412,Symptom
3,clementine,MESH:D002974,Histamine H1 receptor,UniProt:P35367,Itching of skin,MESH:D011537,downregulates,associates,Gene::3269,Compound::DB00283,Symptom::D011537,Symptom
4,clementine,MESH:D002974,Histamine H1 receptor,UniProt:P35367,sneezing,MESH:D012912,downregulates,associates,Gene::3269,Compound::DB00283,Symptom::D012912,Symptom
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Dantrolene,MESH:D003620,Ryanodine receptor 1,UniProt:P21817,Spasticity,MESH:D009128,downregulates,associates,Gene::6261,Compound::DB01219,Symptom::D009128,Symptom
77,Vindesine,MESH:D014751,Tubulin beta-1 chain,UniProt:Q9H4B7,Malignant melanoma,MESH:D008545,downregulates,associates,Gene::81027,Compound::DB00309,Disease::DOID:1909,Disease
78,chlorothiazide,MESH:D002740,Carbonic anhydrase 2,UniProt:P00918,Hypertensive disorder,MESH:D006973,downregulates,associates,Gene::760,Compound::DB00880,Disease::DOID:10763,Disease
79,Phenytoin,MESH:D010672,Sodium channel protein type 5 subunit alpha,UniProt:Q14524,Epilepsy,MESH:D004827,downregulates,associates,Gene::6331,Compound::DB00252,Disease::DOID:1826,Disease
